In [1]:
import json
from collections import defaultdict, OrderedDict
import numpy as np
import os, shutil
import subprocess
import json
from typing import List, Dict, Optional, Any, Tuple
import glob

In [2]:
import textworld
import textworld.gym
import gym

In [3]:
import twutils
from twutils.file_helpers import count_iter_items, parse_gameid, split_gamename
from twutils.twlogic import filter_observables
from twutils.redis_ids import *


In [4]:
if os.path.isdir('/ssdt2b/ftwc'):
    TW_DATA_BASEDIR = '/ssd2tb/ftwc'
else:
    TW_DATA_BASEDIR = '/work2/gstrazds/ftwc'


In [5]:
TW_TRAINING_DIR = f'{TW_DATA_BASEDIR}/games/train/'
TW_VALIDATION_DIR = f'{TW_DATA_BASEDIR}/games/valid/'
TW_TEST_DIR = f'{TW_DATA_BASEDIR}/games/test/'

PTHRU_DIR = f'{TW_DATA_BASEDIR}/playthru_data/'
RANDOM_SEED = 42


In [6]:
import tokenizers
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.pre_tokenizers import Punctuation

from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, Strip, Replace, Sequence


In [7]:
from tokenizers.trainers import UnigramTrainer

In [8]:
special_tokens = ['<PAD>', '<UNK>', '<S>', '</S>', '<bos>', '<eos>', '<NONE>', '<sep>',]
                  
    #  '---Exits---']  #, COMMAND_TOKEN]

                 # '+open', '+closed', '+roasted', '+baked', '+fried', '+raw',
                 # '+sliced', '+diced', '+chopped', '++Carrying:', ]
normalizer = normalizers.Sequence([Strip(), Lowercase()])
pre_tokenizer = Whitespace()

model = tokenizers.models.WordLevel(unk_token='<UNK>')
# model = tokenizers.models.WordPiece()
tokenizer = Tokenizer(model=model)


tokenizer.add_special_tokens(special_tokens)
# tokenizer.add_tokens(['---Exits---'])  #, COMMAND_TOKEN,])
# tokenizer.add_tokens(['frosted-glass'])
# tokenizer.add_tokens(['[UNK]'])  #'++Carrying:', 'hello', '+how', 'today'])
tokenizer.normalizer = normalizer
tokenizer.pre_tokenizer = pre_tokenizer

# filelist = glob.glob(PTHRU_DIR+"miniset/*.pthru")
filelist = glob.glob(PTHRU_DIR+"valid/*.pthru")
filelist.extend( glob.glob(PTHRU_DIR+"test/*.pthru"))
filelist.extend( glob.glob(PTHRU_DIR+"train/*.pthru"))

### glob.glob(PTHRU_DIR+"*.textds"))

# token_strs = [tok for (tok, span) in pre_tokenizer.pre_tokenize_str(str1)]
# print(token_strs)
print(len(filelist))

filelist = sorted(filelist)


#unigram_trainer = tokenizers.trainers.UnigramTrainer()
trainer = tokenizers.trainers.WordLevelTrainer(vocab_size=500, special_tokens=special_tokens)
#trainer = tokenizers.trainers.WordPieceTrainer(vocab_size=100)

tokenizer.train(files=filelist, trainer=trainer)


5176


In [9]:
for f in filelist:
    with open(f, "r") as fin:
        for line in fin:
            if 'start' in line.split():
                print(f, line)

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+cook+cut+drop-be7I6yghQBXCZpO.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+cook+cut-18Excnm6H0DWhpE3.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+cook-67J1t2ykC5BruW67.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+cut+drop-2GmxfyWEhJGMi2DN.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+cut-yQO0UpZoSZagU9Vj.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+take1+cook+cut+drop+go12-BaPLI1E9ID7LtR7M.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+take1+cook+cut+drop+go6-3MdMIP1oiDWPHogg.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+take1+cook+cut+drop+go9-9jPlsD6dtYvmu9Ne.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe1+take1+cook+cut+drop-X711IdgmT336i1KR

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+cut+open-5BgKIV92SlgPfpv8.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+cut-mGPriGPxTD6luB6r.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+drop+go12-y3ORsv9xhkNauK2k.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+drop+go6-KPlEiEP2FXZjFPkN.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+drop+go9-xMajIk9BCdxUPdx.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+drop-vgOGUDj9IQbRtNg3.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+go12-e2yQhMxZcEWjTJnj.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+cook+go6-X9Qkfvbmhgd5HN7B.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/test/tw-cooking-recipe5+take5+c

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-Wq3OUXnkHD7YTm5k.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-XLemUN09c8EEHQd0.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-YbkbfayaCbqaFPZe.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-aZWyhJplIZKpf7pm.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-d19eTraZUP8QH3Oq.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-ednoUDWoT0O6Ue2G.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-mGjEcl3gtNBxiDg3.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe1+cut+open+drop+go9-nRyZCdX3fP2DCyoe.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-reci

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-Lq11TBygCK1OSRo2.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-VyKLhrVbI7NOCWpo.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-ZBGDSyLWsdZ2IGmk.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-jypMHqpRUJRpck6M.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-kaD1uBlbsP1qiWR5.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-kb7GcMM3HaQLFjD1.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-l9ojixBGt92pFx0R.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+cook+cut+open+drop+go9-ovGvum3ytWbZs5Y3.pthru >>>[ start ]<<<

/work2/gstrazds/

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-Z7L8CvEPsO53iKDg.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-ZNn8h87ysvdGFQXl.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-aaLbhEn1IyOoCDl3.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-axM7sgerIeErHoax.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-gX6XTWBvsWYNI6bX.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-kjJ5FQvWt1kmIxg0.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-lR8KUVm5T12kun7y.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe2+take2+cook+cut+open+drop-yvGLIG57syBmSM98.pthru >>>[ start ]<<<



/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-a9bxCbMaC86lcqaE.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-adkbcpqdtRyKiyvb.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-aeWxIMlyFvjnC8bP.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-bmojIYe3Fd5OsjbD.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-mv0pUOWEtRGpFZeQ.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-nGx2ugDgt5mxhLKk.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-oQ1ZIX3gC57XS6MD.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go6-ordPc3MDC8MPfpZJ.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+cook+go9-0K3GiRYghpJDHR2l.pthru >>>[ start ]<<<

/work2/gstrazds/ftw

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go12-oO6lsGvlT5JWu0Gv.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go12-oQoasVW0F7vBuxgq.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go12-omRbhRONFQPlSy2d.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go6-2LVrTOoKTeKnSBWJ.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go6-3ka3sD1GImRaTkd6.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go6-99KPTG9vher0h8Qm.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go6-EM6QSy1MFeoPsayy.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+cut+drop+go6-GVvohbKVhO0gurgj.pthru >>>[ start ]<<<

/work

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-El3WtmLBilmGhvdd.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-G3mKiZElfv5quGJQ.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-Lg6RIeKGHvDZCYP8.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-WlMBTR9JcRp9i3qq.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-Y1mNskbefDMjIp8g.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-Y5MLIpV9UvDDsykx.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-YgkjcYXLCqdDugG.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+cook+open+go12-ZRrjfpo1UoPvu1lJ.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_dat

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop+go9-o8qJuB5pfxmocR6N.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop+go9-peeOHBx9Fm2kF7RN.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop+go9-qELJTDOjf2jdTLJr.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop+go9-vgxgcZVgSZyXub2Y.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop+go9-ygpmHnlt9Lbuxg.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop-2MEOsRXDcVQRu5kd.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop-2jLnho3XF9V3hKNl.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop-3EZ8IDRrfeb9fPR3.pthru >>>[ start ]<<<

/work2/gstrazds/ftwc/playthru_data/train/tw-cooking-recipe3+take3+drop-Br6ES1BkUWlEt8Rk.pthru 

In [10]:
str1 = " ++Carrying: Hello there, +how are   you \n today? "

print(normalizer.normalize_str(str1))
token_strs = [tok for (tok, span) in pre_tokenizer.pre_tokenize_str(str1)]
print(token_strs)

encoding = tokenizer.encode(str1)
print(encoding.tokens)

++carrying: hello there, +how are   you 
 today?
['++', 'Carrying', ':', 'Hello', 'there', ',', '+', 'how', 'are', 'you', 'today', '?']
['++', 'carrying', ':', 'hello', 'there', ',', '+', 'how', 'are', 'you', 'today', '?']


In [11]:
print(filelist[-1])

/work2/gstrazds/ftwc/playthru_data/valid/tw-cooking-recipe3-N1e6TqGOUvo8IvNa.pthru


In [12]:
pthru_str = open(filelist[0], 'r').read()
#print(pthru_str)
pthru_encoding = tokenizer.encode(pthru_str)

In [13]:
# pthru_str = open(filelist[-1], 'r').read()
# #print(pthru_str)
# pthru_encoding = tokenizer.encode(pthru_str)

In [14]:
token_list = pthru_encoding.tokens[:]
print(len(token_list), token_list)

607 ['>>>[', 'start', ']<<<', '---------', 'do', ':', 'find', 'kitchen', ',', 'read', 'cookbook', ',', 'eat', 'meal', '.', '-=', 'kitchen', '=-', 'in', '+', 'open', 'fridge', ':', 'nothing', ';', 'in', '+', 'closed', 'oven', ':', 'unknown', ';', 'on', 'table', ':', 'nothing', ';', 'on', 'counter', ':', 'cookbook', ',', 'knife', ';', 'on', 'stove', ':', 'nothing', ';', 'carrying', ':', '+', 'raw', 'block', 'of', 'cheese', ';', '---------', 'do', ':', 'find', 'kitchen', ',', 'read', 'cookbook', ',', 'eat', 'meal', '.', '>>>[', 'open', 'oven', ']<<<', 'you', 'open', 'the', 'oven', '.', '-=', 'kitchen', '=-', 'in', '+', 'open', 'fridge', ':', 'nothing', ';', 'in', '+', 'open', 'oven', ':', 'nothing', ';', 'on', 'table', ':', 'nothing', ';', 'on', 'counter', ':', 'cookbook', ',', 'knife', ';', 'on', 'stove', ':', 'nothing', ';', 'carrying', ':', '+', 'raw', 'block', 'of', 'cheese', ';', '---------', 'do', ':', 'find', 'kitchen', ',', 'read', 'cookbook', ',', 'eat', 'meal', '.', '>>>[', 'rea

In [15]:
decoded = tokenizer.decode(pthru_encoding.ids, skip_special_tokens=False)
print(decoded)

>>>[ start ]<<< --------- do : find kitchen , read cookbook , eat meal . -= kitchen =- in + open fridge : nothing ; in + closed oven : unknown ; on table : nothing ; on counter : cookbook , knife ; on stove : nothing ; carrying : + raw block of cheese ; --------- do : find kitchen , read cookbook , eat meal . >>>[ open oven ]<<< you open the oven . -= kitchen =- in + open fridge : nothing ; in + open oven : nothing ; on table : nothing ; on counter : cookbook , knife ; on stove : nothing ; carrying : + raw block of cheese ; --------- do : find kitchen , read cookbook , eat meal . >>>[ read cookbook ]<<< you read the recipe --------- acquire : block of cheese ; --------- do : slice block of cheese , fry block of cheese , prepare meal ; -= kitchen =- in + open fridge : nothing ; in + open oven : nothing ; on table : nothing ; on counter : cookbook , knife ; on stove : nothing ; carrying : + raw block of cheese ; --------- acquire : block of cheese ; --------- do : slice block of cheese ,

In [16]:
# help(tokenizer)

In [17]:
vocab = tokenizer.get_vocab(with_added_tokens=True)
slist = [(vocab[k], k) for k in vocab.keys()]
print(sorted(slist))
tokenizer.save('ftwc_tokenizer_new.json', pretty=True)


[(0, '<PAD>'), (1, '<UNK>'), (2, '<S>'), (3, '</S>'), (4, '<bos>'), (5, '<eos>'), (6, '<NONE>'), (7, '<sep>'), (8, ';'), (9, ','), (10, '+'), (11, ':'), (12, 'raw'), (13, 'on'), (14, 'open'), (15, 'red'), (16, 'nothing'), (17, 'to'), (18, '---------'), (19, 'pepper'), (20, 'potato'), (21, 'meal'), (22, 'in'), (23, '.'), (24, 'yellow'), (25, 'unknown'), (26, 'do'), (27, 'kitchen'), (28, 'cookbook'), (29, 'onion'), (30, 'door'), (31, 'bell'), (32, '-='), (33, ']<<<'), (34, '=-'), (35, '>>>['), (36, 'you'), (37, 'prepare'), (38, 'chop'), (39, 'carrying'), (40, 'exits'), (41, 'oven'), (42, 'the'), (43, 'acquire'), (44, 'fridge'), (45, 'knife'), (46, 'table'), (47, 'counter'), (48, 'stove'), (49, 'east'), (50, 'north'), (51, 'west'), (52, 'south'), (53, 'go'), (54, 'apple'), (55, 'hot'), (56, 'chicken'), (57, 'white'), (58, 'purple'), (59, 'read'), (60, 'eat'), (61, 'pork'), (62, 'banana'), (63, 'slice'), (64, 'fry'), (65, 'orange'), (66, 'of'), (67, 'block'), (68, 'cheese'), (69, 'dice'), 

In [18]:
tokenizer2 = Tokenizer.from_file('/ssd2tb/ftwc/ftwc_tokenizer.json')
vocab2 = tokenizer2.get_vocab(with_added_tokens=True)
print(len(vocab), len(vocab2))
for i in range(max(len(vocab), len(vocab2))):
    tok1 = tokenizer.id_to_token(i)
    if tok1 is None:
        tok1 = 'NONE'
    tok2 = tokenizer2.id_to_token(i)
    if tok2 is None:
        tok2 = 'NONE'
    if tok2 != tok1:
        tok1 = "* "+tok1
    else:
        tok1 = "  "+tok1
    print(f"{i}:{tok1:9s}\t{tok2}")
print()

155 157
0:  <PAD>  	<PAD>
1:  <UNK>  	<UNK>
2:  <S>    	<S>
3:  </S>   	</S>
4:* <bos>  	<|>
5:* <eos>  	<NONE>
6:* <NONE> 	<bos>
7:* <sep>  	<eos>
8:* ;      	<sep>
9:* ,      	;
10:  +      	+
11:  :      	:
12:* raw    	,
13:* on     	raw
14:* open   	on
15:* red    	open
16:  nothing	nothing
17:  to     	to
18:* ---------	---
19:* pepper 	in
20:* potato 	red
21:* meal   	unknown
22:* in     	pepper
23:* .      	door
24:* yellow 	]<<<
25:* unknown	-=
26:* do     	>>>[
27:* kitchen	=-
28:* cookbook	.
29:* onion  	you
30:* door   	potato
31:* bell   	carrying
32:* -=     	exits
33:* ]<<<   	kitchen
34:* =-     	oven
35:* >>>[   	cookbook
36:* you    	the
37:* prepare	fridge
38:* chop   	knife
39:* carrying	table
40:* exits  	counter
41:* oven   	yellow
42:* the    	stove
43:* acquire	east
44:* fridge 	north
45:* knife  	west
46:* table  	south
47:* counter	go
48:* stove  	onion
49:* east   	bell
50:* north  	closed
51:* west   	meal
52:* south  	hot
53:* go     	apple
54:* apple  	pat

In [19]:
pthru_encoding2 = tokenizer2.encode(pthru_str)
token_list2 = pthru_encoding2.tokens[:]
print(len(token_list2))
print(token_list2)
assert token_list2 == token_list

607
['>>>[', 'start', ']<<<', '---------', 'do', ':', 'find', 'kitchen', ',', 'read', 'cookbook', ',', 'eat', 'meal', '.', '-=', 'kitchen', '=-', 'in', '+', 'open', 'fridge', ':', 'nothing', ';', 'in', '+', 'closed', 'oven', ':', 'unknown', ';', 'on', 'table', ':', 'nothing', ';', 'on', 'counter', ':', 'cookbook', ',', 'knife', ';', 'on', 'stove', ':', 'nothing', ';', 'carrying', ':', '+', 'raw', 'block', 'of', 'cheese', ';', '---------', 'do', ':', 'find', 'kitchen', ',', 'read', 'cookbook', ',', 'eat', 'meal', '.', '>>>[', 'open', 'oven', ']<<<', 'you', 'open', 'the', 'oven', '.', '-=', 'kitchen', '=-', 'in', '+', 'open', 'fridge', ':', 'nothing', ';', 'in', '+', 'open', 'oven', ':', 'nothing', ';', 'on', 'table', ':', 'nothing', ';', 'on', 'counter', ':', 'cookbook', ',', 'knife', ';', 'on', 'stove', ':', 'nothing', ';', 'carrying', ':', '+', 'raw', 'block', 'of', 'cheese', ';', '---------', 'do', ':', 'find', 'kitchen', ',', 'read', 'cookbook', ',', 'eat', 'meal', '.', '>>>[', 'rea

In [20]:
decoded2 = tokenizer2.decode(pthru_encoding2.ids, skip_special_tokens=False)
print(decoded2)

>>>[ start ]<<< --------- do : find kitchen , read cookbook , eat meal . -= kitchen =- in + open fridge : nothing ; in + closed oven : unknown ; on table : nothing ; on counter : cookbook , knife ; on stove : nothing ; carrying : + raw block of cheese ; --------- do : find kitchen , read cookbook , eat meal . >>>[ open oven ]<<< you open the oven . -= kitchen =- in + open fridge : nothing ; in + open oven : nothing ; on table : nothing ; on counter : cookbook , knife ; on stove : nothing ; carrying : + raw block of cheese ; --------- do : find kitchen , read cookbook , eat meal . >>>[ read cookbook ]<<< you read the recipe --------- acquire : block of cheese ; --------- do : slice block of cheese , fry block of cheese , prepare meal ; -= kitchen =- in + open fridge : nothing ; in + open oven : nothing ; on table : nothing ; on counter : cookbook , knife ; on stove : nothing ; carrying : + raw block of cheese ; --------- acquire : block of cheese ; --------- do : slice block of cheese ,

In [21]:
decoded == decoded2

True

In [22]:
pthrus_all = open(f'{PTHRU_DIR}/mingpt-valid-all.pthru', 'r').read()
encoded_all = tokenizer.encode(pthrus_all)

In [23]:
len(encoded_all.tokens)

157275

In [24]:
str_out = tokenizer.decode(encoded_all.ids, skip_special_tokens=False)
print(str_out[0:200])

>>>[ start ]<<< --------- do : find kitchen , read cookbook , eat meal . -= kitchen =- in + open fridge : nothing ; in + closed oven : unknown ; on table : nothing ; on counter : cookbook ; on stove :


In [25]:
"<UNK>" in str_out

False

In [26]:
tokenizer.token_to_id('>>>['), tokenizer.token_to_id(']<<<')


(35, 33)

In [27]:
vocab_dict = tokenizer.get_vocab(with_added_tokens=False)
print(len(vocab_dict))
vocab_dict2 = tokenizer.get_vocab(with_added_tokens=True)
print(len(vocab_dict2))
print(tokenizer.get_vocab_size(with_added_tokens=False))
print(tokenizer.get_vocab_size(with_added_tokens=True))  #seems to be wrong!

155
155
155
163


In [28]:
print(vocab_dict)

{'do': 26, ':': 11, 'pick': 128, '<UNK>': 1, 'tomato': 116, '<sep>': 7, 'open': 14, 'bbq': 111, 'closed': 70, 'toolbox': 145, 'fried': 76, 'roast': 71, 'plain': 86, '.': 23, 'carrying': 39, 'potato': 20, 'the': 42, 'east': 49, '</S>': 3, 'bell': 31, 'cheese': 68, 'you': 36, 'eat': 60, 'floor': 79, 'chair': 117, 'too': 124, 'commercial': 141, 'black': 95, 'orange': 65, 'an': 153, 'with': 109, '<eos>': 5, 'prepare': 37, 'nothing': 16, 'west': 51, 'red': 15, 'go': 53, 'grill': 77, 'wing': 87, "'": 121, 'lettuce': 149, 'flour': 105, 'street': 127, '<NONE>': 6, '<S>': 2, 'breast': 148, 'and': 147, 'take': 84, 'chopped': 93, 'workbench': 150, 'fridge': 44, 'unknown': 25, 'a': 119, 'things': 122, '---------': 18, 'many': 123, 'onion': 29, 'oil': 101, '<PAD>': 0, 'acquire': 43, '<bos>': 4, 'oven': 41, 'shed': 134, 'counter': 47, 'start': 114, 'exits': 40, 'meal': 21, 'recipe': 113, 'milk': 136, 'in': 22, 'bed': 140, 'parsley': 73, 'peanut': 154, 'screen': 106, 'driveway': 118, ';': 8, ']<<<': 

In [29]:
print(tokenizer.id_to_token(len(vocab_dict)-1))
print(tokenizer.id_to_token(tokenizer.get_vocab_size(with_added_tokens=True)-1))

peanut
None


In [30]:
tokenizer.token_to_id('>>>[')

35